# **Filter & Merging**

Combine all three data sources and trim some columns we don't need.

In [1]:
# Libraries

from postal.expand import expand_address
import pandas as pd
import pickle

In [2]:
# directories

# input
dataDir = '../../data/external/'
icDir = dataDir + 'iowa-city-police/'
uiDir = dataDir + 'ui-police/'

# output
processDir = '../../data/processed/'
icOut = processDir + 'iowa-city-police/'
uiOut = processDir + 'ui-police/'

# Dataframes
citeDF = pd.read_pickle((icOut + 'icpd-cites.pickle'))
arrDF = pd.read_pickle((icOut + 'arrests-icpd.pickle'))
dockDF = pd.read_pickle((uiOut + 'ui-police-docket.pickle'))

### Columns, Variables

In [3]:
citeDF.columns

Index([u'lName', u'fName', u'homeAddr', u'homeCity', u'dob', u'timestamp',
       u'charge', u'incAddr', u'incDate', u'incFormAddr'],
      dtype='object')

In [4]:
arrDF.columns

Index([u'name', u'dob', u'homeAddr', u'homeCity', u'timestamp', u'incDate',
       u'incAddr', u'incAct', u'charge', u'incFormAddr'],
      dtype='object')

In [5]:
dockDF.columns

Index([u'incDate', u'name', u'age', u'homeAddr', u'incAddr', u'date2',
       u'chargeCode', u'charge', u'incTime', u'homeCity', u'homeState',
       u'homeZip'],
      dtype='object')

We want to keep the following variables, and we'll work our way towards this schema:

```
Arrest Record
---------------------------------
    name (last, first middle)
    age
    date/timestamp (unix)
    incAddr
    charge
```

In [6]:
# Name
citeDF['name'] = citeDF['lName'] + ', ' + citeDF['fName']

In [7]:
# dob parsing
citeDF.dropna(subset=['dob'], inplace=True)
citeDF['dob'] = pd.to_datetime(citeDF.dob, infer_datetime_format=True, exact=False, errors='coerce')

In [8]:
# Age in years, 365 days in a year
def calcAge(dob, inc):
    
    # error handling
    if ((type(dob) or type(inc)) != pd.tslib.Timestamp):
        return
    
    td = (inc - dob).days
    return td // 365

# Naive implementation has trouble with missing values
#citeDF.age = citeDF.incDate - citeDF.dob
# citeDF.age = citeDF.age.apply(lambda x: x.days // 365)


citeDF['age'] = citeDF.apply(lambda x: calcAge(x.dob, x.incDate), axis=1)

In [ ]:
citeDF.head(2)

In [ ]:
# timestamp
### COME BACK TO TIMESTAMP
all.data$arrtime <- (((as.numeric(all.data$arrtime))-(as.numeric(all.data$arryear)*1000000))/60) %% 24
citeDF.timestamp = 

Now we move onto the Iowa City Police Department arrests. We only need...

- [x]Age calculation
- [ ] Timestamp to UTC calc

In [9]:
# age
arrDF['age'] = arrDF.apply(lambda x: calcAge(x.dob, x.incDate), axis=1)

In [ ]:
# timestamp


University of Iowa Police docket, renaming to make it easier when we filter cols.

In [10]:
dockDF.rename(columns={'incAddr': 'incFormAddr', 'incTime': 'timestamp'}, inplace=True)

---

Now we filter the columns, so that we have a slimmer dataset thats normalized. We'll also add the type of criminal activity (source) in a column.

In [11]:
# add that column to each df
citeDF['source'] = 'c'
arrDF['source'] = 'a'
dockDF['source'] = 'd'

In [12]:
# filter columns
slimCols = ['name', 'age', 'incDate', 'timestamp', 'charge', 'incFormAddr', 'source']
# and combine!
combDF = pd.concat([citeDF[slimCols], arrDF[slimCols], dockDF[slimCols]])

In [14]:
# drop records without a charge, because that's (in the end), what we're interested in.

before = combDF.shape[0]
print ('{} total records'.format(before))

combDF.dropna(subset=['charge'], inplace=True)

print ('{} after dropping those without a charge,\n{} deleted'\
      .format(combDF.shape[0], before - combDF.shape[0]))

139698 total records
136108 total records after dropping those without a charge,
3590 deleted


In [16]:
# SAVE THAT ISH OUT
combDF.to_pickle((processDir + 'combined-activity(no ts).pickle'))
combDF.to_csv((processDir + 'combined-activity(no ts).csv'), index=False)